[Reference](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0)

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
os.chdir('gdrive/My Drive/Colab Notebooks')
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content
/content/gdrive/My Drive/Colab Notebooks


In [3]:
import pandas as pd

In [4]:
papers = pd.read_csv('./papers.csv')
# Print head
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


# Data Cleaning

In [5]:
# Remove the columns
papers = papers.drop(columns=['id', 'title', 'abstract', 
                              'event_type', 'pdf_name', 'year'], axis=1)
# sample only 10 papers - for demonstration purposes
papers = papers.sample(10)
# Print out the first rows of papers
papers.head()

,paper_text
2959,Particle-based Variational Inference\nfor Cont...
5650,Deep Learning without Poor Local Minima\n\nKen...
2112,Learning Influence among Interacting\nMarkov C...
878,The Manhattan World Assumption:\nRegularities ...
4286,Bayesian Model Comparison and Backprop Nets\n\...


In [6]:
# Load the regular expression library
import re
# Remove punctuation
papers['paper_text_processed'] = papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

2959    particle-based variational inference\nfor cont...
5650    deep learning without poor local minima\n\nken...
2112    learning influence among interacting\nmarkov c...
878     the manhattan world assumption:\nregularities ...
4286    bayesian model comparison and backprop nets\n\...
Name: paper_text_processed, dtype: object

In [7]:
%%time
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['particle', 'based', 'variational', 'inference', 'for', 'continuous', 'systems', 'alexander', 'ihler', 'dept', 'of', 'computer', 'science', 'univ', 'of', 'california', 'irvine', 'ihler', 'icsuciedu', 'andrew', 'frank', 'dept', 'of', 'computer', 'science', 'univ', 'of', 'california', 'irvine', 'ajfrank', 'icsuciedu', 'padhraic', 'smyth', 'dept', 'of', 'computer', 'science', 'univ', 'of', 'california', 'irvine', 'smyth', 'icsuciedu', 'abstract', 'since', 'the', 'development', 'of', 'loopy', 'belief', 'propagation', 'there', 'has', 'been', 'considerable', 'work', 'on', 'advancing', 'the', 'state', 'of', 'the', 'art', 'for', 'approximate', 'inference', 'over', 'distributions', 'defined', 'on', 'discrete', 'random', 'variables', 'improvements', 'include', 'guarantees', 'of', 'convergence', 'approximations', 'that', 'are', 'provably', 'more', 'accurate', 'and', 'bounds', 'on', 'the', 'results', 'of', 'exact', 'inference', 'however', 'extending', 'these', 'methods', 'to', 'continuous', 'val

# Bi-grams and Tri-grams

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['particle', 'base', 'variational', 'inference', 'continuous', 'system', 'univ', 'icsuciedu', 'univ', 'univ', 'icsuciedu', 'abstract', 'development', 'loopy', 'considerable', 'work', 'advance', 'state', 'art', 'approximate', 'inference', 'distribution', 'define', 'discrete', 'random', 'variable', 'improvement', 'include', 'guarantee', 'convergence', 'approximation', 'provably', 'accurate', 'bound', 'result', 'exact', 'inference', 'however', 'extend', 'method', 'continuous', 'value', 'system', 'lag', 'several', 'method', 'develop', 'belief_propagation', 'system', 'continuous', 'value', 'recent', 'advance', 'discrete', 'variable', 'incorporated', 'context', 'extend', 'recently', 'propose', 'particle', 'base', 'provide', 'general', 'framework', 'adapt', 'discrete', 'algorithm', 'inference', 'continuous', 'system', 'result', 'algorithm', 'behave', 'similarly', 'purely', 'discrete', 'counterpart', 'extend', 'benefit', 'advance', 'inference', 'technique', 'continuous', 'domain', 'introducti

# Data Transformation: Corpus and Dictionary

In [13]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 5), (10, 3), (11, 2), (12, 2), (13, 1), (14, 7), (15, 3), (16, 1), (17, 1), (18, 20), (19, 1), (20, 1), (21, 8), (22, 8), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 6), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 5), (37, 4), (38, 6), (39, 1), (40, 20), (41, 16), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 11), (59, 1), (60, 4), (61, 1), (62, 2), (63, 20), (64, 4), (65, 4), (66, 1), (67, 1), (68, 5), (69, 2), (70, 3), (71, 1), (72, 7), (73, 1), (74, 3), (75, 1), (76, 1), (77, 1), (78, 3), (79, 9), (80, 2), (81, 4), (82, 2), (83, 1), (84, 2), (85, 1), (86, 4), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 2), (94, 1), (95, 2), (96, 2), (97, 1), (98, 6), (99, 3), (100, 1), (101, 1), (102, 3), (103, 1), (104, 1), (105, 2), (106, 2), (107, 1), (108, 1), (109, 3), (11

# Base Model

In [14]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [15]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.002*"deep" + 0.002*"model" + 0.002*"loss" + 0.001*"function" + '
  '0.001*"assumption" + 0.001*"network" + 0.001*"layer" + 0.001*"work" + '
  '0.001*"nonlinear" + 0.001*"result"'),
 (1,
  '0.044*"model" + 0.043*"player" + 0.041*"influence" + 0.018*"level" + '
  '0.017*"state" + 0.017*"team" + 0.015*"meeting" + 0.014*"individual" + '
  '0.013*"use" + 0.012*"action"'),
 (2,
  '0.024*"model" + 0.014*"inference" + 0.012*"method" + 0.011*"use" + '
  '0.011*"cnf" + 0.010*"variable" + 0.009*"output" + 0.009*"function" + '
  '0.008*"distribution" + 0.008*"crf"'),
 (3,
  '0.018*"set" + 0.015*"number" + 0.014*"model" + 0.013*"datum" + '
  '0.013*"feature" + 0.012*"training" + 0.011*"dataset" + 0.010*"algorithm" + '
  '0.010*"evidence" + 0.010*"parameter"'),
 (4,
  '0.001*"model" + 0.001*"use" + 0.001*"value" + 0.001*"method" + '
  '0.001*"function" + 0.001*"set" + 0.001*"input" + 0.001*"result" + '
  '0.001*"feature" + 0.001*"output"'),
 (5,
  '0.001*"model" + 0.001*"inference" + 0.001

In [16]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.38207800869311986


There is the model hyperparameters to determine for good performance of our model:
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

In [17]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [19]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


  0%|          | 0/270 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)

100%|██████████| 270/270 [08:02<00:00,  1.79s/it]


In [21]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.01)

In [22]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"model" + 0.014*"deep" + 0.012*"function" + 0.010*"assumption" + '
  '0.010*"image" + 0.009*"edge" + 0.009*"loss" + 0.009*"point" + '
  '0.008*"result" + 0.008*"follow"'),
 (1,
  '0.029*"model" + 0.012*"use" + 0.008*"method" + 0.008*"number" + '
  '0.007*"variable" + 0.007*"player" + 0.007*"distribution" + 0.007*"set" + '
  '0.007*"influence" + 0.007*"feature"')]


In [24]:
!pip install pyLDAvis
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

     |████████████████████████████████| 1.6MB 8.8MB/s 
     |████████████████████████████████| 552kB 46.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=9f419aa86ef642c74c13cf77348fbe31189a034d8fcf874655fb7c9d1d7263db
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=d3e8f41d95b73c428270298ed9997ac28cf78fab75bbc39b1f10608b69815022
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.132756  0.0       1        1  73.927399
0     -0.132756  0.0       2        1  26.072594, topic_info=           Term        Freq       Total Category  logprob  loglift
820        deep   60.000000   60.000000  Default  30.0000  30.0000
52   assumption   54.000000   54.000000  Default  29.0000  29.0000
918        loss   39.000000   39.000000  Default  28.0000  28.0000
310       image   53.000000   53.000000  Default  27.0000  27.0000
273    function  102.000000  102.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
769      assume   22.903175   51.815010   Topic2  -5.2239   0.5279
436     network   26.861172   86.736633   Topic2  -5.0644   0.1721
614        show   20.902838   72.790726   Topic2  -5.3152   0.0966
715         use   25.001595  172.604279   Topic2  -5.1362  -0.5878
716       value   20.919582   94.754318   Topic2  -5.3144  -0.1663

[170 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1078      1  1.000829      action
746       2  1.004618  activation
18        1  0.844700   algorithm
18        2  0.157486   algorithm
21        1  0.804741        also
...     ...       ...         ...
2312      1  1.000330       vi_vj
1069      2  1.004340          wh
733       1  0.386283        work
733       2  0.618053        work
1985      2  1.002831          yk

[191 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])